In [ ]:
import requests
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
import os

class WeatherRecommender:
    def __init__(self, api_key, city, user_prefs=None, user_weight=70, water_drank_today=0):
        self.api_key = api_key
        self.city = city
        self.weather_data = {}
        self.uv_index = None
        self.forecast_data = {}
        self.user_prefs = user_prefs if user_prefs else {
            "cold_sensitive": True,
            "likes_outdoors": True,
            "prefers_detailed_tips": True
        }
        self.user_weight = user_weight
        self.water_drank_today = water_drank_today

    def fetch_weather_data(self):
        print("📡 Fetching weather data...")
        try:
            url = f"https://api.openweathermap.org/data/2.5/weather?q={self.city}&appid={self.api_key}&units=metric"
            response = requests.get(url)
            response.raise_for_status()
            self.weather_data = response.json()
        except requests.RequestException as e:
            print("❌ Error fetching weather data:", e)
            self.weather_data = None

    def fetch_uv_index(self):
        print("🌞 Checking UV Index...")
        try:
            lat = self.weather_data['coord']['lat']
            lon = self.weather_data['coord']['lon']
            url = f"http://api.openweathermap.org/data/2.5/uvi?lat={lat}&lon={lon}&appid={self.api_key}"
            response = requests.get(url)
            response.raise_for_status()
            self.uv_index = response.json()['value']
        except Exception as e:
            print("⚠️ UV Index unavailable:", e)
            self.uv_index = None

    def fetch_forecast_data(self):
        print("🔮 Fetching forecast data...")
        try:
            url = f"https://api.openweathermap.org/data/2.5/forecast?q={self.city}&appid={self.api_key}&units=metric"
            response = requests.get(url)
            response.raise_for_status()
            self.forecast_data = response.json()
        except Exception as e:
            print("⚠️ Could not fetch forecast data:", e)
            self.forecast_data = {}

    def get_time_of_day(self):
        try:
            timezone_offset = self.weather_data.get("timezone", 0)
            utc_now = datetime.now(timezone.utc)
            local_time = utc_now + timedelta(seconds=timezone_offset)
            hour = local_time.hour

            if hour < 12:
                return "morning"
            elif hour < 18:
                return "afternoon"
            else:
                return "evening"
        except Exception as e:
            print("⚠️ Could not calculate local time:", e)
            return "day"

    def classify_uv_risk(self):
        if self.uv_index is None:
            return "unknown"
        elif self.uv_index < 3:
            return "low"
        elif self.uv_index < 6:
            return "moderate"
        elif self.uv_index < 8:
            return "high"
        else:
            return "very high"

    def calculate_remaining_hydration(self):
        if not self.weather_data:
            return None

        temp = self.weather_data['main']['temp']
        wind_speed = self.weather_data['wind']['speed']
        uv_risk = self.classify_uv_risk()

        base_need = self.user_weight * 35  # ml
        extra = 0

        if temp > 25:
            extra += 500
        if uv_risk in ["high", "very high"]:
            extra += 300
        if wind_speed >= 6:
            extra += 300

        total_need = base_need + extra
        remaining = max(0, total_need - self.water_drank_today)

        return {
            "total_required": total_need,
            "drank": self.water_drank_today,
            "remaining": remaining
        }

    def generate_advice(self):
        if not self.weather_data:
            return [("⚠️", "No weather data available.")]

        main = self.weather_data['main']
        weather = self.weather_data['weather'][0]['main']
        desc = self.weather_data['weather'][0]['description']
        wind = self.weather_data['wind']
        time_of_day = self.get_time_of_day()

        advice = []

        temp = main['temp']
        humidity = main['humidity']
        wind_speed = wind['speed']

        advice.append(("📊", f"Current Temperature: {temp}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s"))

        if temp < 0:
            advice.append(("🧊 URGENT", "Extreme cold! Stay indoors if possible."))
        elif temp < 5:
            advice.append(("🧥", "Wear thermal clothing, coat, gloves, and scarf."))
        elif 5 <= temp < 15:
            advice.append(("🧣", "Wear a warm sweater or light jacket."))
        elif 15 <= temp < 25:
            advice.append(("👕", "Comfortable range. Light clothing is fine."))
        else:
            advice.append(("🔥", "High heat! Wear breathable clothes and stay hydrated."))

        if "Rain" in weather:
            advice.append(("🌧️", "Carry a waterproof umbrella and wear boots."))
            if wind_speed > 10:
                advice.append(("🌬️", "Avoid using umbrellas due to high wind."))
        elif "Snow" in weather:
            advice.append(("❄️", "Dress in layers. Wear snow boots and gloves."))
        elif "Thunderstorm" in weather:
            advice.append(("⛈️ URGENT", "Stay indoors. Lightning and heavy rain expected."))
        elif "Clear" in weather:
            if self.user_prefs["likes_outdoors"]:
                advice.append(("🌞", f"It's clear and sunny. Great for a walk in the {time_of_day}."))
        elif "Clouds" in weather and humidity > 85:
            advice.append(("☁️", "Humid and overcast. Might rain later."))

        if wind_speed >= 12:
            advice.append(("💨", "Very windy! Avoid cycling or flying umbrellas."))
        elif wind_speed >= 6:
            advice.append(("🍃", "Mild wind. Secure loose items."))

        if self.uv_index is not None:
            uv_risk = self.classify_uv_risk()
            if uv_risk == "high":
                advice.append(("🧴", "UV is high. Wear sunglasses and sunscreen."))
            elif uv_risk == "very high":
                advice.append(("🔥 URGENT", "Extreme UV risk. Avoid prolonged sun exposure."))

        if self.user_prefs["cold_sensitive"] and temp < 15:
            advice.append(("⚠️", "As you're cold-sensitive, dress warmer than average."))
        if not self.user_prefs["likes_outdoors"] and "Rain" in weather:
            advice.append(("🚫", "Better to cancel any outdoor plans."))

        return advice

    def display_advice(self, advice_list):
        print(f"\n📍 Recommendations for {self.city.title()} ({self.get_time_of_day()}):\n")
        for tag, msg in advice_list:
            print(f"{tag} {msg}")

        hydration = self.calculate_remaining_hydration()
        if hydration:
            print(f"\n🚰 Hydration Advice:")
            print(f"💧 Total Required: {hydration['total_required']:.0f} ml")
            print(f"✅ Drank: {hydration['drank']:.0f} ml")
            print(f"🧃 Remaining Today: {hydration['remaining']:.0f} ml")

        if self.forecast_data:
            print("\n📅 3-Hour Forecast Snapshot:")
            try:
                for item in self.forecast_data["list"][:3]:
                    time = datetime.fromtimestamp(item['dt'], tz=timezone.utc) + timedelta(
                        seconds=self.forecast_data['city']['timezone']
                    )
                    temp = item['main']['temp']
                    weather = item['weather'][0]['description'].capitalize()
                    wind = item['wind']['speed']
                    humidity = item['main']['humidity']
                    pop = item.get('pop', 0) * 100
                    print(f"⏰ {time.strftime('%Y-%m-%d %H:%M')} | 🌡️ {temp}°C | ☁️ {weather} | 💨 {wind} m/s | 💧 {humidity}% | 🌧️ {pop:.0f}% chance")
            except Exception as e:
                print("⚠️ Error displaying forecast:", e)

    def run(self):
        self.fetch_weather_data()
        if self.weather_data:
            self.fetch_uv_index()
            self.fetch_forecast_data()
        advice_list = self.generate_advice()
        self.display_advice(advice_list)

if __name__ == "__main__":
    load_dotenv() 
    API_KEY = os.getenv('API_KEY')  

    CITY = input("Enter your city: ")
    try:
        weight = float(input("Enter your weight in kg (default 70): ") or "70")
    except ValueError:
        weight = 70
    try:
        water_today = float(input("How much water have you drunk today (ml)? ") or "0")
    except ValueError:
        water_today = 0

    preferences = {
        "cold_sensitive": True,
        "likes_outdoors": True,
        "prefers_detailed_tips": True
    }

    wr = WeatherRecommender(api_key=API_KEY, city=CITY, user_prefs=preferences,
                            user_weight=weight, water_drank_today=water_today)
    wr.run()


📡 Fetching weather data...
🌞 Checking UV Index...
🔮 Fetching forecast data...

📍 Recommendations for Lahore (morning):

📊 Current Temperature: 29.99°C, Humidity: 66%, Wind Speed: 2.06 m/s
🔥 High heat! Wear breathable clothes and stay hydrated.
🔥 URGENT Extreme UV risk. Avoid prolonged sun exposure.

🚰 Hydration Advice:
💧 Total Required: 2200 ml
✅ Drank: 5500 ml
🧃 Remaining Today: 0 ml

📅 3-Hour Forecast Snapshot:
⏰ 2025-09-19 05:00 | 🌡️ 29.03°C | ☁️ Scattered clouds | 💨 2.37 m/s | 💧 68% | 🌧️ 0% chance
⏰ 2025-09-19 08:00 | 🌡️ 29.66°C | ☁️ Few clouds | 💨 2.66 m/s | 💧 64% | 🌧️ 0% chance
⏰ 2025-09-19 11:00 | 🌡️ 33.89°C | ☁️ Clear sky | 💨 2.2 m/s | 💧 46% | 🌧️ 0% chance
